# ctext API 활용

### ctext 및 필요한 패키지 설치

In [ ]:
!pip install ctext # https://pypi.org/project/ctext/

In [ ]:
from ctext import *
setlanguage("en") # ctext 언어 설정 (zh는 중국어)
# setlanguage("zh")
# setremap("gb")  # 간체를 원하면 주석 해제
import pandas as pd
import re

### ctext API 써보기

##### 데이터베이스 제목 불러오기 (gettexttitles)
https://api.ctext.org/gettexttitles

In [ ]:
# title : 원서 제목, urn : url 주소
gettexttitles()

In [ ]:
# 총 문헌 개수
len(gettexttitles()['books'])

##### 웹 링크에서 고유 주소 가져오기 (readlink)

In [ ]:
readlink("http://ctext.org/mengzi") # 맹자 고유번호

##### 고유주소로 텍스트 딕셔너리 형태로 추출 (gettext)
https://api.ctext.org/gettext?urn=ctp:analects/xue-er

In [ ]:
gettext("ctp:analects/xue-er") # 논어 - 학이편, 문단 단위로 분절된 상태로 추출

In [ ]:
gettext("ctp:analects") # 논어 전편

##### 여러 단위로 텍스트 가져오기

In [ ]:
# 논어 전체 텍스트 리스트로 가져오기
gettextaschapterlist("ctp:analects")

In [ ]:
# 논어 편 단위로 나눠서 텍스트 가져오기
gettextasobject("ctp:analects")

In [ ]:
# 문단 단위로 가져오기 - 논어 전편
gettextasparagraphlist("ctp:analects")

In [ ]:
# 문단 단위로 가져오기 - 논어 학이편
gettextasparagraphlist("ctp:analects/xue-er")

In [ ]:
# 문단 분리 없이 통으로 가져오기 - 논어 학이편
gettextasstring("ctp:analects/xue-er")

##### 텍스트 고유 정보

In [ ]:
# 논어 고유 정보
gettextinfo("ctp:analects")

##### 제목(title) 단위 검색

In [ ]:
# 제목에 '論語'가 포함된 텍스트 리스트 추출
searchtexts("論語")

### ctext 텍스트 수집 실습
목표 : 자신이 수집을 원하는 텍스트 범위를 정해 엑셀 형태로 저장해보기  
*예시 : 논어 텍스트 전편을 문단(문장) 단위로 추출해보기*

In [ ]:
# 고유 주소 확보
gettext("ctp:analects")['subsections']

In [ ]:
gettext("ctp:analects/xue-er")

In [ ]:
# 데이터 프레임으로 만들기
pd.DataFrame(gettext("ctp:analects/xue-er"))

In [ ]:
# 데이터 프레임을 엑셀로 저장
pd.DataFrame(gettext("ctp:analects/xue-er")).to_excel('xue_er.xlsx')

##### gettext 함수를 for문으로 반복실행

In [ ]:
# 논어 장단위 고유주소 리스트 만들기
analects_list = gettext("ctp:analects")['subsections']
analects_list

In [ ]:
df = pd.DataFrame() # empty dataframe 생성
for url in analects_list:
  chapter =  pd.DataFrame(gettext(url))
  df = df.append(chapter,ignore_index=True)

In [ ]:
df

In [ ]:
# column 순서 변경
df = df[['title','fulltext']]
df

In [ ]:
# # \n\n을 구분점으로 문단으로 텍스트 분리해서 자료를 만드는 방법(결과 같음)
# df['fulltext'].str.split('\n\n').explode('para')
# df = df.assign(fulltext=df['fulltext'].str.split('\n\n')).explode('fulltext')
# # 빈칸인 행 삭제
# df = df[df['fulltext']!='']
# df = df.reset_index(drop=True)
# df

In [ ]:
# 엑셀로 저장
df.to_excel('analects.xlsx', index=None)

##### 대화문만 추출해보기
*홑낫표(「」) 사이에 있는 문장*

In [ ]:
sentece_test = df.loc[9,'fulltext']
sentece_test

In [ ]:
# 정규표현식으로 홑낫표 사이 문장만 추출하기
re.findall('「(.+?)」',sentece_test)

In [ ]:
df['sentence'] = df['fulltext'].str.findall('「(.+?)」')
df['sentence']

In [ ]:
# 문장 리스트 -> 문자열
df = df.explode('sentence')
df

In [ ]:
# reset index
df = df.reset_index(drop=True)
df

In [ ]:
# 대화가 없는 부분
df[pd.isna(df['sentence'])]

In [ ]:
# 엑셀로 저장
df.to_excel('analects_sent.xlsx', index=None)

##### 추가 아이디어
*   구두점(。?!)을 기준으로 문장을 다시 한번 더 나눌 수도 있다.
*   대화문의 자와 화자 표기는 曰을 기준으로 1차 추출후에 사람이 직접 확인하며 나눌 수 있음.



##### 특정 단어가 포함된 문단/문장

In [ ]:
# 仁 이 들어간 문장
df[df['sentence'].str.contains('仁')]

In [ ]:
ren = df[df['sentence'].str.contains('仁',na=False)]
ren

##### 기술통계량

In [ ]:
# 장별 대화수
df['title'].value_counts()

In [ ]:
# 장별 대화수 (비율)
df['title'].value_counts(normalize=True)

In [ ]:
# 仁이 들어간 장별 대화수
ren['title'].value_counts()